In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_mistralai import ChatMistralAI
from typing import List
from enum import Enum

from pydantic import BaseModel, Field
import json
from typing import List

In [3]:
#model = init_chat_model("ministral-8b-latest", model_provider="mistralai", temperature=0)
model = init_chat_model("mistral-large-latest", model_provider="mistralai", temperature=0)

In [4]:
class Period(str, Enum):
    """Enum for periods of life."""
    CHILDHOOD = "childhood"
    ADOLESCENCE = "adolescence"
    YOUNG_ADULTHOOD = "young_adulthood"
    ADULTHOOD = "adulthood"
    OLD_AGE = "old_age"

class Person(BaseModel):
    id: int = Field(description="The unique identifier for the person")
    name: str = Field(description="The name of the person")
    relationship: str = Field(description="The relationship of the person to the user")
    age: int = Field(description="The age of the person")
    occupation: str = Field(description="The occupation of the person")
    hobbies: List[str] = Field(default_factory=list, description="The hobbies of the person")
    description: str = Field(description="A description of the person")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

class Memory(BaseModel):
    people: List[int] = Field(min_items=1, decription="A list of IDs of people involved in the memory")
    place: List[str] = Field(min_items=1, default_factory=list, description="A list of places involved in the memory")
    period: List[str] = Field(min_items=1, default_factory=list)
    emotion: List[str] = Field(min_items=1, default_factory=list)
    description: str = Field(description="A description of the memory")

    def __str__(self):
        return json.dumps(self.dict(), indent=2)
    
class LifeSummary(BaseModel):
    name: str = Field(description="The name of the user")
    age: int = Field(description="The age of the user")
    occupation: str = Field(description="The occupation of the user")
    acquaintances: List[Person] = Field(min_items=5, default_factory=list, description="A list of acquaintances of the user (friend, colleague,  family)")
    memories: List[Memory] = Field(default_factory=list)

    def __str__(self):
        return json.dumps(self.dict(), indent=2)

In [6]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage("Tu est un chatbot qui génère des données de synthèse sur les souvenirs sur toute la vie d'une personne agée."),
        ("user", """Invente cinq souvenirs. pour une personne dont voici la description :
            Nom : {name}
            Age : {age}
            Profession : {occupation} 
         Rédige les souvenirs à la première personne,
         en utilisant le vocabulaire et les émotions d'une personne âgée.
         Chaque souvenir doit être spécifique et détaillé, en incluant la référence (id) des personnes impliquées, les lieux, la période de la vie et les émotions ressenties."""),
    ]
)
chain = prompt_template | model.with_structured_output(LifeSummary)

In [ ]:
life_summary = chain.invoke({"name": "Monique", "age": "75", "occupation": "Ancienne Assistante sociale"})

In [10]:
life_summary.model_dump()

{'name': 'Monique',
 'age': 75,
 'occupation': 'Ancienne Assistante sociale',
 'acquaintances': [{'id': 1,
   'name': 'Jean',
   'relationship': 'Mari',
   'age': 78,
   'occupation': 'Retraité',
   'hobbies': ['Jardinage', 'Lecture'],
   'description': 'Mon mari depuis 50 ans, un homme patient et aimant.'},
  {'id': 2,
   'name': 'Sophie',
   'relationship': 'Fille',
   'age': 45,
   'occupation': 'Professeur',
   'hobbies': ['Peinture', 'Randonnée'],
   'description': 'Ma fille aînée, une femme indépendante et passionnée.'},
  {'id': 3,
   'name': 'Pierre',
   'relationship': 'Fils',
   'age': 42,
   'occupation': 'Ingénieur',
   'hobbies': ['Football', 'Cuisine'],
   'description': 'Mon fils cadet, un homme dévoué et toujours prêt à aider.'},
  {'id': 4,
   'name': 'Marie',
   'relationship': "Amie d'enfance",
   'age': 75,
   'occupation': 'Retraitée',
   'hobbies': ['Tricot', 'Voyages'],
   'description': "Mon amie d'enfance, toujours là pour moi dans les bons et les mauvais momen